<a href="https://colab.research.google.com/github/Iamsdt/UdacityPyTorch/blob/master/src/notebook/PretrainedModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

In [0]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

import torch
from torchvision import datasets, transforms, models

import helper

In [5]:
import requests, zipfile, io

url = 'https://s3.amazonaws.com/content.udacity-data.com/nd089/Cat_Dog_data.zip'  

r = requests.get(url)
z = zipfile.ZipFile(io.BytesIO(r.content))
dir = z.extractall("Cat_Dog_data")

print("done")

done


In [4]:
# create train and test loader
data_dir = 'Cat_Dog_data/Cat_Dog_data'

# Define transforms for the training data and testing data
train_transforms = transforms.Compose([
    transforms.RandomRotation(30),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()
])

test_transforms = transforms.Compose([
    transforms.Resize(255),
    transforms.CenterCrop(224),
    transforms.ToTensor()
])

# Pass transforms in here, then run the next cell to see how the transforms look
train_data = datasets.ImageFolder(data_dir + '/train', transform=train_transforms)
test_data = datasets.ImageFolder(data_dir + '/test', transform=test_transforms)

trainloader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=64)

#size of train_data
print(len(train_data))

22500


In [5]:
# load a pretrained model
model = models.densenet121(pretrained = True)

/usr/local/lib/python3.6/dist-packages/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)


In [6]:
# model current classifer
print(model.classifier)

Linear(in_features=1024, out_features=1000, bias=True)


In [0]:
# Freeze the parameters
for param in model.parameters():
  param.required_grad = False

In [0]:
from collections import OrderedDict
from torch import nn
classifier = nn.Sequential(OrderedDict([
                ('fc1', nn.Linear(1024, 500)),
                ('relu', nn.ReLU()),
                ('fc2', nn.Linear(500, 2)),
                ('output', nn.LogSoftmax(dim=1))
                ]))

# replace the classifer
model.classifier = classifier

In [9]:
# check the classifier
print(model.classifier)

Sequential(
  (fc1): Linear(in_features=1024, out_features=500, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=500, out_features=2, bias=True)
  (output): LogSoftmax()
)


In [10]:
from torch import optim
# move model to cuda (GPU)
# model.to("cuda")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

criterion = nn.NLLLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=0.003)

#move tensor to default device
model.to(device)

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplac

In [11]:
# train model
epochs = 1
steps = 0
running_loss = 0
print_every = 5

for epoch in range(epochs):
    
    for inputs, labels in trainloader:
        steps += 1
        # Move input and label tensors to the default device
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        logps = model.forward(inputs)
        loss = criterion(logps, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        
        if steps % print_every == 0:
            test_loss = 0
            accuracy = 0
            model.eval()
            with torch.no_grad():
                for inputs, labels in testloader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    logps = model.forward(inputs)
                    batch_loss = criterion(logps, labels)
                    
                    test_loss += batch_loss.item()
                    
                    # Calculate accuracy
                    ps = torch.exp(logps)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
                    
            print(f"Epoch {epoch+1}/{epochs}.. "
                  f"Train loss: {running_loss/print_every:.3f}.. "
                  f"Test loss: {test_loss/len(testloader):.3f}.. "
                  f"Test accuracy: {accuracy/len(testloader):.3f}")
            running_loss = 0
            model.train()

Epoch 1/1.. Train loss: 1.057.. Test loss: 0.585.. Test accuracy: 0.693
Epoch 1/1.. Train loss: 0.669.. Test loss: 0.289.. Test accuracy: 0.858
Epoch 1/1.. Train loss: 0.490.. Test loss: 0.280.. Test accuracy: 0.882
Epoch 1/1.. Train loss: 0.348.. Test loss: 0.224.. Test accuracy: 0.895
Epoch 1/1.. Train loss: 0.269.. Test loss: 0.109.. Test accuracy: 0.964
Epoch 1/1.. Train loss: 0.220.. Test loss: 0.071.. Test accuracy: 0.979
Epoch 1/1.. Train loss: 0.204.. Test loss: 0.063.. Test accuracy: 0.979
Epoch 1/1.. Train loss: 0.170.. Test loss: 0.061.. Test accuracy: 0.981
Epoch 1/1.. Train loss: 0.142.. Test loss: 0.067.. Test accuracy: 0.976
Epoch 1/1.. Train loss: 0.237.. Test loss: 0.066.. Test accuracy: 0.975
Epoch 1/1.. Train loss: 0.208.. Test loss: 0.068.. Test accuracy: 0.974
Epoch 1/1.. Train loss: 0.167.. Test loss: 0.054.. Test accuracy: 0.981
Epoch 1/1.. Train loss: 0.170.. Test loss: 0.058.. Test accuracy: 0.979
Epoch 1/1.. Train loss: 0.173.. Test loss: 0.054.. Test accuracy

KeyboardInterrupt: ignored

In [0]:
print(model)

In [0]:
torch.save(model.state_dict(),"pretrainedCatDog.pth")

In [0]:
# send pic to model
model.eval()

images,labels = next(iter(trainloader))

# take single img
img = images[0]
# conver to 1D
img = img.view(1,-1)

with torch.no_grad():
    output = model.forward(img)

ps = torch.exp(output)
helper.view_classify(img.view(1,28,28),ps,version='Fashion')

In [1]:
# obtain one batch of test images
dataiter = iter(testloader)
images, labels = dataiter.next()

# Move to cpu
model.cpu()
  
# get sample outputs
output = model(images)
# convert output probabilities to predicted class
_, preds = torch.max(output, 1)
# prep images for display
images = images.numpy()

# plot the images in the batch, along with predicted and true labels
fig = plt.figure(figsize=(25, 4))
for idx in np.arange(4):
    ax = fig.add_subplot(2, 20/2, idx+1, xticks=[], yticks=[])
    ax.imshow(np.squeeze(images[idx]), cmap='gray')
    ax.set_title("{} ({})".format(str(preds[idx].item()), str(labels[idx].item())),
                 color=("green" if preds[idx]==labels[idx] else "red"))

NameError: ignored